In [2]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup

In [3]:
question_df = pd.read_csv("Questions.csv", encoding='latin1')
tag_df = pd.read_csv("Tags.csv")
question_df.shape, tag_df.shape

((85085, 6), (244228, 2))

In [4]:
question_df.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,6,5.0,2010-07-19T19:14:44Z,272,The Two Cultures: statistics vs. machine learn...,"<p>Last year, I read a blog post from <a href=..."
1,21,59.0,2010-07-19T19:24:36Z,4,Forecasting demographic census,<p>What are some of the ways to forecast demog...
2,22,66.0,2010-07-19T19:25:39Z,208,Bayesian and frequentist reasoning in plain En...,<p>How would you describe in plain English the...
3,31,13.0,2010-07-19T19:28:44Z,138,What is the meaning of p values and t values i...,<p>After taking a statistics course and then t...
4,36,8.0,2010-07-19T19:31:47Z,58,Examples for teaching: Correlation does not me...,"<p>There is an old saying: ""Correlation does n..."


In [5]:
tag_df.head()

,Id,Tag
0,1,bayesian
1,1,prior
2,1,elicitation
3,2,distributions
4,2,normality


In [6]:
question_df = question_df[['Id', 'Title', 'Body']]
question_df.head()

,Id,Title,Body
0,6,The Two Cultures: statistics vs. machine learn...,"<p>Last year, I read a blog post from <a href=..."
1,21,Forecasting demographic census,<p>What are some of the ways to forecast demog...
2,22,Bayesian and frequentist reasoning in plain En...,<p>How would you describe in plain English the...
3,31,What is the meaning of p values and t values i...,<p>After taking a statistics course and then t...
4,36,Examples for teaching: Correlation does not me...,"<p>There is an old saying: ""Correlation does n..."


In [7]:
question_df['Body'] = question_df['Body'].apply(lambda x: BeautifulSoup(x).get_text())
question_df['Body'] = question_df['Body'].apply(lambda x: re.sub('[^a-zA-z]', ' ', x))
question_df['Body'] = question_df['Body'].apply(lambda x: x.lower())
question_df['Body'] = question_df['Body'].apply(lambda x: x.split())
question_df['Body'] = question_df['Body'].apply(lambda x: ' '.join(x))

In [8]:
question_df.sample(6)

,Id,Title,Body
15613,147930,How to determine study weights for a random-ef...,i want to do a meta analysis with lme where fo...
31878,87478,Implementation for Class Association Rule mini...,it seems to me that all the implementation for...
15021,202215,Lag variables for forecasting weekly from dail...,i am working on a daily response variable as p...
35712,76440,"Plot 3 density plots in the same graph, showin...",i have obtained from a paper next results vari...
51342,41260,Joint probability and Gaussian copula,i have \pr a \ and \pr b \ where a and b are t...
48395,92890,Compare distributions with uneven and small sa...,i will be performing my analysis in r but any ...


In [9]:
tag_df = tag_df.groupby('Id').apply(lambda x: x['Tag'].values).reset_index(name='tags')
tag_df.head()

/tmp/ipython-input-2564490889.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tag_df = tag_df.groupby('Id').apply(lambda x: x['Tag'].values).reset_index(name='tags')


,Id,tags
0,1,"[bayesian, prior, elicitation]"
1,2,"[distributions, normality]"
2,3,"[software, open-source]"
3,4,"[distributions, statistical-significance]"
4,6,[machine-learning]


In [10]:
df = pd.merge(question_df, tag_df)

In [11]:
df.head()

,Id,Title,Body,tags
0,6,The Two Cultures: statistics vs. machine learn...,last year i read a blog post from brendan o co...,[machine-learning]
1,21,Forecasting demographic census,what are some of the ways to forecast demograp...,"[forecasting, population, census]"
2,22,Bayesian and frequentist reasoning in plain En...,how would you describe in plain english the ch...,"[bayesian, frequentist]"
3,31,What is the meaning of p values and t values i...,after taking a statistics course and then tryi...,"[hypothesis-testing, t-test, p-value, interpre..."
4,36,Examples for teaching: Correlation does not me...,there is an old saying correlation does not me...,"[correlation, teaching]"


In [12]:
freq = {}
for i in df['tags']:
  for j in i:
    if j in freq.keys():
      freq[j] += 1
    else:
      freq[j] = 1

freq = dict(sorted(freq.items(), key=lambda x: x[1] , reverse=True))

In [13]:
common_tags = list(freq.keys())[:20]
common_tags

['r',
 'regression',
 'machine-learning',
 'time-series',
 'probability',
 'hypothesis-testing',
 'self-study',
 'distributions',
 'logistic',
 'classification',
 'correlation',
 'statistical-significance',
 'bayesian',
 'anova',
 'normal-distribution',
 'multiple-regression',
 'mixed-model',
 'clustering',
 'neural-networks',
 'mathematical-statistics']

In [14]:
x = []
y = []

for i in range(len(df['tags'])):
  temp = []
  for j in df['tags'][i]:
    if j in common_tags:
      temp.append(j)

  if len(temp) > 1:
    x.append(df['Body'][i])
    y.append(temp)

In [15]:
len(x)

19689

In [16]:
y[:20]

[['r', 'time-series'],
 ['regression', 'distributions'],
 ['r', 'clustering'],
 ['time-series', 'neural-networks'],
 ['probability', 'bayesian'],
 ['distributions', 'normal-distribution'],
 ['distributions',
  'probability',
  'hypothesis-testing',
  'mathematical-statistics'],
 ['self-study', 'mathematical-statistics'],
 ['distributions', 'bayesian'],
 ['correlation', 'self-study'],
 ['regression', 'correlation', 'mathematical-statistics'],
 ['hypothesis-testing', 'self-study'],
 ['r', 'regression', 'time-series'],
 ['hypothesis-testing', 'mathematical-statistics'],
 ['r', 'time-series', 'self-study'],
 ['probability', 'hypothesis-testing'],
 ['r', 'regression'],
 ['r', 'mixed-model'],
 ['r', 'regression', 'anova'],
 ['regression', 'logistic', 'multiple-regression']]

In [17]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)

print(y)

[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 0]]


In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=True)

In [19]:
y_test[0]

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer

X_tokenizer = Tokenizer()
X_tokenizer.fit_on_texts(X_train)
len(X_tokenizer.word_index)

35803

In [21]:
tresh = 3
cnt = 0

for key, val in X_tokenizer.word_counts.items():
  if val >= tresh:
    cnt += 1
cnt

17443

In [22]:
X_tokenizer = Tokenizer(num_words=cnt, oov_token='unk')
X_tokenizer.fit_on_texts(X_train)
len(X_tokenizer.word_index)

35804

In [23]:
max_len = 100

X_train_seq = X_tokenizer.texts_to_sequences(X_train)
X_test_seq = X_tokenizer.texts_to_sequences(X_test)

In [24]:
from keras.preprocessing.sequence import pad_sequences

X_train_seq = pad_sequences(X_train_seq, padding='post', maxlen=max_len)
X_test_seq = pad_sequences(X_test_seq, padding='post', maxlen=max_len)
X_test_seq

array([[ 165,   18,   82, ...,   32, 2253,   81],
       [  22,  187,    5, ...,   63,    3,  205],
       [ 129,   64,  149, ...,    0,    0,    0],
       ...,
       [  44, 5852,   11, ...,   57,   52,   19],
       [6652, 2518, 2248, ..., 2518, 4761, 6652],
       [   4,   15,   78, ...,   14,   22, 3558]], dtype=int32)

In [25]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *

In [26]:
x_voc_size = X_tokenizer.num_words + 1

In [58]:
model = Sequential()
model.add(Embedding(input_dim=x_voc_size, output_dim=50, input_shape=(max_len, ), mask_zero=True))
model.add(GRU(128))
model.add(Dense(units=20, activation='sigmoid'))

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 100, 50)        │       872,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 128)            │        69,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 20)             │         2,580 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 943,900 (3.60 MB)

 Trainable params: 943,900 (3.60 MB)

 Non-trainable params: 0 (0.00 B)

In [59]:
from tensorflow.keras import metrics

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=[
        metrics.Precision(name='precision'),
        metrics.Recall(name='recall')
    ]
)


In [60]:
checkpoint = ModelCheckpoint('best_weighs.keras', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [61]:
model.fit(X_train_seq, y_train, validation_data=(X_test_seq, y_test), batch_size=64, epochs=20, callbacks=[checkpoint])

Epoch 1/20
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step - loss: 0.4045 - precision: 0.1073 - recall: 0.0237
Epoch 1: val_loss improved from inf to 0.32007, saving model to best_weighs.keras
247/247 ━━━━━━━━━━━━━━━━━━━━ 62s 239ms/step - loss: 0.4042 - precision: 0.1073 - recall: 0.0237 - val_loss: 0.3201 - val_precision: 0.8679 - val_recall: 0.0053
Epoch 2/20
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - loss: 0.3161 - precision: 0.7120 - recall: 0.0153
Epoch 2: val_loss improved from 0.32007 to 0.29456, saving model to best_weighs.keras
247/247 ━━━━━━━━━━━━━━━━━━━━ 59s 237ms/step - loss: 0.3161 - precision: 0.7118 - recall: 0.0155 - val_loss: 0.2946 - val_precision: 0.6087 - val_recall: 0.1586
Epoch 3/20
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step - loss: 0.2772 - precision: 0.6790 - recall: 0.1689
Epoch 3: val_loss improved from 0.29456 to 0.25283, saving model to best_weighs.keras
247/247 ━━━━━━━━━━━━━━━━━━━━ 84s 246ms/step - loss: 0.2771 - precision: 0.6791 - recall: 0.1690 - val_loss: 

In [62]:
model.load_weights('best_weighs.keras')

In [63]:
pred_prob = model.predict(X_test_seq)
np.round(pred_prob[0], 2)

124/124 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step


array([0.  , 0.02, 0.28, 0.02, 0.01, 0.  , 0.01, 0.01, 0.86, 0.02, 0.  ,
       0.  , 0.63, 0.  , 0.02, 0.06, 0.07, 0.04, 0.01, 0.11],
      dtype=float32)

In [64]:
threshold = np.arange(0, 0.5, 0.01)
threshold

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49])

In [65]:
def classify(pred_pro, tsh):
  y_pred_seq = []

  for i in pred_pro:
    temp = []
    for j in i:
      if j>=tsh:
        temp.append(1)
      else:
        temp.append(0)
    y_pred_seq.append(temp)

  return y_pred_seq

In [66]:
from sklearn.metrics import *

score = []
y_true = np.array(y_test)

for thresh in threshold:
  y_pred_seq = classify(pred_prob, thresh)
  y_pred = np.array(y_pred_seq)
  score.append(f1_score(y_true, y_pred, average='micro'))

In [67]:
opt = threshold[score.index(max(score))]
opt

np.float64(0.33)

In [68]:
y_true = np.array(y_test)
y_pred = classify(pred_prob, opt)
y_pred = np.array(y_pred)

y_pred[0], y_true[0]

(array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [70]:
y_pred[1], y_true[1]

(array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]))

In [71]:
y_pred[23], y_true[23]

(array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]))

In [72]:
y_pred[678], y_true[678]

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]))

In [69]:
precision = precision_score(y_true, y_pred, average='micro', zero_division=0)
recall = recall_score(y_true, y_pred, average='micro', zero_division=0)
f1 = f1_score(y_true, y_pred, average='micro', zero_division=0)

print("Overall Performance Summary:")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")

Overall Performance Summary:
Precision: 0.6103
Recall:    0.6037
F1-score:  0.6070


In [ ]:
#SIMPLE RNN

# Overall Performance Summary:
# Precision: 0.3965
# Recall:    0.5033
# F1-score:  0.4436

In [ ]:
#LSTM

# Overall Performance Summary:
# Precision: 0.5726
# Recall:    0.5967
# F1-score:  0.5844

# (array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
#  array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [ ]:
#GRU

# Overall Performance Summary:
# Precision: 0.6103
# Recall:    0.6037
# F1-score:  0.6070

# (array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
#  array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))